## 커맨드 디자인 패턴

커맨드 패턴은 객체가 특정 기능을 바로 수행하거나 나중에 트리거할 때 필요한 모든 정보를 캡슐화하는 행동 패턴이다. 캡슐화하는 정보는 다음과 같다.

* 메소드명
* 메소드를 소유하는 객체
* 메소드 인자

프린트 스풀러는 종이 크기, 가로/세로, 부수 등의 설정을 Command 객체에 저장한다. 인쇄를 요청하면 스풀러는 Command 객체의 execute() 함수를 호출하고 설정대로 인쇄한다.

## 커맨드 패턴 구성 요소

커맨드 패턴은 Command와 Receiver, Invoker, Client 클래스로 구성된다.

* Command 객체는 Receiver 객체에 대해 알고 있으며 Receiver 객체의 함수를 호출
* Receiver 함수의 인자는 Command에 저장돼 있다.
* Invoker는 명령을 수행한다.
* Client는 Command 객체를 생성하고 Receiver를 정한다.

커맨드 패턴의 목적

* 요청을 객체 속에 캡슐화
* 클라이언트의 다양한 요청을 매개변수화
* 유청을 큐에 저장
* 객체지향 콜백을 지원

커맨드 패턴의 적합한 상황

* 수행할 명령에 따라 객체를 변수화할 때
* 요청을 큐에 저장하고 각기 다른 시점에 수행해야 하는 경우
* 작은 단위의 연산을 기반으로 하는 상위 연산을 만들 때

In [3]:
class Wizard():
    def __init__(self, src, rootdir):
        self.choices = []
        self.rootdir = rootdir
        self.src = src
    
    def preferences(self, command):
        self.choices.append(command)
    
    def execute(self):
        for choice in self.choices:
            if list(choice.values())[0]:
                print("Copy binaries --", self.src, " to ", self.rootdir)
            else:
                print("No Operation")

# 클라이언트 코드
wizard = Wizard('python3.6.gzip', '/usr/bin/')

# 사용자가 원하는 파이썬 선택
wizard.preferences({'python': True})
wizard.preferences({'java': False})
wizard.preferences({'c': True})
wizard.execute()

Copy binaries -- python3.6.gzip  to  /usr/bin/
No Operation
Copy binaries -- python3.6.gzip  to  /usr/bin/


* Command: 연산을 수행할 인터페이스를 정의
* ConcreteCommand: Receiver 객체와 연산간 바인딩을 정의
* Client: ConcreteCommand 객체를 생성하고 Receiver를 설정
* Invoker: ConcreteCommand에 수행을 요청
* Receiver: 요청에 관련된 연산을 관리

클라이언트는 특정 연산을 요청하고 Invoker는 요청을 받아 캡슐화해 큐에 넣는다. ConcreteCommand 클래스는 이 요청을 책임지고 Receiver에 수행을 맡긴다.

In [5]:
from abc import ABCMeta, abstractmethod

class Command(metaclass = ABCMeta):
    def __init__(self, recv):
        self.recv = recv
        
    def execute(self):
        pass
    
class ConcreteCommand(Command):
    def __init__(self, recv):
        self.recv = recv
    
    def execute(self):
        self.recv.action()

class Receiver:
    def action(self):
        print("Receiver Action")

class Invoker:
    def command(self, cmd):
        self.cmd = cmd
    
    def execute(self):
        self.cmd.execute()

recv = Receiver()
cmd = ConcreteCommand(recv)
invoker = Invoker()
invoker.command(cmd)
invoker.execute()

Receiver Action


## 커맨드 패턴의 실제 활용 사례

증권거래소를 커맨드 패턴으로 구현해본다. 일반적으로 고객이 주식을 직접 사거나 팔지 않고 에이전트나 브로커가 고객과 거래소 사이에서 중개자 역할을 한다. 중개사는 고객의 요청을 받아 책임지고 거래소에 전달해 처리한다.

### 클래스 설계

* Order 클래스는 Command 객체를 나타낸다.
* Order 클래스는 파이썬의 추상 기본 클래스인 인터페이스이며 Concrete Command는 이를 기반으로 세부 로직을 구현한다.
* execute() 메소드는 ConcreteCommand 클래스가 Order 클래스를 실행하는 추상 메소드다.

다음처럼 Order 추상 클래스와 execute() 추상 메소드를 정의한다.

In [6]:
from abc import ABCMeta, abstractmethod

class Order(metaclass=ABCMeta):
    
    @abstractmethod
    def execute(self):
        pass

ConcreteCommand를 나타내는 몇 가지 클래스를 작성

* BuyStockOrder와 SellStockOrder는 Order 인터페이스를 구현하는 구상 클래스다.
* 각 클래스는 증권 거래 시스템 객체를 사용해 주식을 매수 또는 매도한다.
* 각 클래스의 execute() 메소드는 주식 객체를 통해 주식을 매수 또는 매도한다.

Order 인터페이스를 구현하는 구상 클래스를 작성한다.

In [7]:
class BuyStockOrder(Order):
    def __init__(self, stock):
        self.stock = stock
    
    def execute(self):
        self.stock.buy()

class SellStockOrder(Order):
    def __init__(self, stock):
        self.stock = stock
    
    def execute(self):
        self.stock.sell()

다음은 증권 거래 시스템을 구현

* StockTrade 클래스는 Receiver 객체
* ConcreteCommand 객체가 생성한 주문을 처리하는 여러 메소드를 정의
* Receiver에 정의된 buy()와 sell() 메소드는 BuyStockOrder와 SellStockOrder 클래스가 주식을 매수 또는 매도할 떄 호출

StockTrade 클래스 작성

In [11]:
class StockTrade:
    def buy(self):
        print("매수")
    
    def sell(self):
        print("매도")

Invoker 구현

* Agent 클래스는 Invoker
* Agent는 클라이언트와 StockExchange 객체 사이의 중개자이며 클라이언트의 주문을 처리
* Agent에는 큐를 나타내는 \__orderQueue 리스트형 데이터 멤버가 있다. 모든 신규 주문건은 큐에 추가
* placeOrder() 메소드는 주문을 큐에 넣고 처리까지 담당

Invoker 역할을 하는 Agent 클래스를 작성

In [12]:
class Agent:
    def __init__(self):
        self.__orderQueue = []
    
    def placeOrder(self, order):
        self.__orderQueue.append(order)
        order.execute()

위를 바탕으로 클라이언트 구현을 생각해보자.

* 클라이언트는 우선 StockTrade 클래스를 Receiver로 지정한다.
* BuyStockOrder와 SellStockOrder(ConcreteCommand)는 StockTarde 객체에 대해 거래를 요청해 주문을 생성
* Invoker 객체는 Agent 클래스를 인터페이스화할 떄 생성된다.
* Agent 클래스의 placeOrder() 메소드는 클라이언트의 요청을 주문한다.

클라이언트 구현

In [13]:
stock = StockTrade()
buyStock = BuyStockOrder(stock)
sellStock = SellStockOrder(stock)

agent = Agent()
agent.placeOrder(buyStock)
agent.placeOrder(sellStock)

매수
매도


클라우드 기반 애플리케이션에서 자주 발생하는 두 가지 상황을 살펴보자.

* 리두 또는 롤백
    * 리두 또는 롤백 기능은 두 가지 구현 방법이 있다.
    * 파일시스템이나 메모리에 스냅샷을 생성하고 롤백이 필요할 때 해당 스냅샷 상태로 되돌린다.
    * 커맨드 패턴을 사용할 경우 커맨드를 순서대로 저장하고 리두가 필요할 떄 저장된 명령을 순차적으로 실행한다.

* 비동기 작업 수행
    * 분산 환경에서 코어 서비스에 요청이 몰리지 않도록 작업을 비동기로 수행하는 경우가 많다.
    * 커맨드 패턴의 Invoker 객체는 모든 요청을 큐에 저장하고 순차적으로 Receiver 객체에 보내어 메인 스레드로부터 독립적으로 수행한다.
    
## 커맨드 패턴의 장단점

장점

* 작업을 요청하는 클래스와 실제로 작업을 수행하는 클래스를 분리
* 큐에 커맨드를 순서대로 저장
* 기존 코드를 수정하지 않고 새로운 커맨드를 쉽게 추가할 수 있다.
* 커맨드 패턴으로 롤백 시스템을 구현할 수 있다. 예로 들었던 인스톨 위저드에 롤백 메소드를 쉽게 추가할 수 있다.

단점

* 클래스와 객체가 많다. 개발자는 신중하게 클래스를 작성해야 한다.
* 모든 작업이 독립적인 ConcreteCommand 클래스이므로 구현 및 유지보수해야 하는 클래스가 많다.